In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Event_log.txt', sep='\t', encoding='cp1251')

In [3]:
df['Event end'] = pd.to_datetime(df['Event end'])

In [4]:
df = df.sort_values(by='Event end')


In [5]:
cases = df.groupby(['CaseID'])['Activity'].apply(lambda x: x.sum())


In [6]:
cases = cases.reset_index()

In [7]:
casesun = cases.drop_duplicates(subset=['Activity'])

In [8]:
casesun['Newcount']=casesun['Activity'].map(cases['Activity'].value_counts())

/Users/iliyashafirov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
caseval = casesun['CaseID']

In [10]:
arrcase = df.groupby(['CaseID'])['Activity'].apply(list)

In [11]:
arrcase = arrcase.reset_index()

In [13]:
caseval = np.array(caseval)

In [14]:
arrcaseun = arrcase.loc[arrcase['CaseID'].isin(caseval)]

In [17]:
arrcaseun['cluster'] = 0

/Users/iliyashafirov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
arrcaseun['unique'] = arrcaseun['Activity'].agg(pd.unique)

/Users/iliyashafirov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
arrcaseun['numactivities'] = arrcaseun['unique'].apply(lambda x: len(x))

/Users/iliyashafirov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
arrcaseun['reps'] = casesun['Newcount']

/Users/iliyashafirov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
cur = arrcaseun[arrcaseun['cluster'] == 0]['numactivities']

In [24]:
from copy import deepcopy
i = 0
therearenans = False
while (len(cur) != 0):
    i = i + 1
    distances = np.zeros((len(cur),2))
    centers = np.zeros(2)
    flag = True
    zero = cur.mean()
    centers[0] = zero
    s = cur - cur.mean()
    center = s.idxmax(s.apply(lambda x: np.linalg.norm(x)))
    centers[1] = cur[center]
    centers_old = np.zeros(centers.shape) 
    clusterser = pd.Series([])
    print(len(cur))
    if (therearenans == True):
        print('hello1')
        break
    while flag == True:
        distances[:,0] = (cur - centers[0]).apply(lambda x: np.linalg.norm(x))
        distances[:,1] = (cur - centers[1]).apply(lambda x: np.linalg.norm(x))
        clusters = np.argmin(distances, axis = 1)
        clusterser = pd.Series(clusters)
        clusterser.index = cur.index
        centers_old = deepcopy(centers)
        centers[0] = np.mean(cur[clusters == 0], axis=0)
        centers[1] = np.mean(cur[clusters == 1], axis=0)
        error = np.linalg.norm(centers - centers_old)
        print(centers_old)
        print(centers)
        if(np.isnan(centers).any() == True):
            flag = False
            therearenans = True
            print('hello')
        print(flag)
        if (error == 0):
            flag = False
        print('flag')
        print(flag)
    new = clusterser[clusterser == 1]
    arrcaseun['cluster'][new.index] = i
    cur = arrcaseun[arrcaseun['cluster'] == 0]['numactivities']
    print(len(cur))
        
        
            
        
        

3143
[11.07254216 23.        ]
[10.83857236 19.1011236 ]
True
flag
True
[10.83857236 19.1011236 ]
[10.03125   16.3371869]
True
flag
True
[10.03125   16.3371869]
[ 9.40828924 15.38628571]
True
flag
True
[ 9.40828924 15.38628571]
[ 8.8225641  14.75020956]
True
flag
True
[ 8.8225641  14.75020956]
[ 8.16346749 14.14725131]
True
flag
True
[ 8.16346749 14.14725131]
[ 8.16346749 14.14725131]
True
flag
False


/Users/iliyashafirov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/iliyashafirov/anaconda3/lib/python3.6/site-packages/pandas/core/series.py:831: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
/Users/iliyashafirov/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


1615
1615
[ 8.16346749 11.        ]
[ 6.87439614 10.4637931 ]
True
flag
True
[ 6.87439614 10.4637931 ]
[ 6.36526946 10.08846154]
True
flag
True
[ 6.36526946 10.08846154]
[ 6.36526946 10.08846154]
True
flag
False
835
835
[6.36526946 8.        ]
[5.77687296 8.        ]
True
flag
True
[5.77687296 8.        ]
[5.2575406 7.5470297]
True
flag
True
[5.2575406 7.5470297]
[5.2575406 7.5470297]
True
flag
False
431
431
[5.2575406 6.       ]
[4.42364532 6.        ]
True
flag
True
[4.42364532 6.        ]
[4.42364532 6.        ]
True
flag
False
203
203
[4.42364532 5.        ]
[3.55555556 5.        ]
True
flag
True
[3.55555556 5.        ]
[3.55555556 5.        ]
True
flag
False
81
81
[3.55555556 4.        ]
[2.56 4.  ]
True
flag
True
[2.56 4.  ]
[2.56 4.  ]
True
flag
False
25
25
[2.56 3.  ]
[1.9 3. ]
True
flag
True
[1.9 3. ]
[1.9 3. ]
True
flag
False
10
10
[1.9 2. ]
[1. 2.]
True
flag
True
[1. 2.]
[1. 2.]
True
flag
False
1
1
[1. 1.]
[ 1. nan]
hello
False
flag
False
1
1
hello1


/Users/iliyashafirov/anaconda3/lib/python3.6/site-packages/pandas/core/series.py:837: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)


In [27]:
arrcaseun = arrcaseun.drop(cur.index, axis = 0)

In [29]:
maxnumclust = max(arrcaseun['cluster'])

In [30]:
sum(arrcaseun['reps'][arrcaseun['cluster'] == 1])

1817

In [31]:
qorclust = []

In [32]:
i = 1
while i <= maxnumclust:
    clust_i = arrcaseun['reps'][arrcaseun['cluster'] == i]
    print(sum(clust_i))
    qorclust.append(sum(clust_i))
    print(i)
    i += 1

1817
1
1370
2
1136
3
743
4
2087
5
540
6
83
7
73
8


In [33]:
qormin = min(qorclust)
qormax = max(qorclust)

In [34]:
def f1(qormin, qormax, qor):
    return (qor - qormin)/(qormax-qormin)

In [35]:
T = []

In [36]:
i = 2
prev = set(arrcaseun['unique'][arrcaseun['cluster'] == 1].apply(pd.Series).stack().value_counts().index)
T.append(len(prev))
while i <= maxnumclust:
    cur = arrcaseun['unique'][arrcaseun['cluster'] == i].apply(pd.Series).stack().value_counts()
    unelems = set(cur.index)
    newelems = unelems - prev
    T.append(len(newelems))
    prev = prev | newelems
    i += 1

In [37]:
arrcaseun['vis'] = 0

In [38]:
arrcaseun['vis'][arrcaseun['cluster'] == 1] = 1

/Users/iliyashafirov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
def g1(T_min, T_max, t):
    return (t - T_min)/(T_max-T_min)

In [40]:
Tmin = min(T)
Tmax = max(T)

In [41]:
i = 2
visclust = []
visvert = []
visclust.append(qorclust[0])
visvert.append(T[0])
while((f1(qormin, qormax, qorclust[i-1]) + g1(Tmin, Tmax, T[i-1])) >(f1(qormin, sum(qorclust), sum(visclust)) + g1(Tmin, sum(T), sum(visvert)))):
    arrcaseun['vis'][arrcaseun['cluster'] == i] = 1
    visclust.append(qorclust[i-1])
    visvert.append(visvert[i-1])
    i+=1
    

In [44]:
import graphviz as gv


In [43]:
graph = gv.Digraph(format="png")
graph.node_attr.update(color='lightblue2', style='filled')

1528

In [98]:
for i in (arrcaseun['Activity'].index):
    arrcaseun['Activity'][i] = [w.replace(':', '') for w in arrcaseun['Activity'][i]]

/Users/iliyashafirov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [114]:
import graphviz as gv

path = arrcaseun['Activity'][]

graph = gv.Digraph(format="png")
graph.node_attr.update(color='lightblue2', style='filled')

# uncomment for high resolution picture
# graph.attr(dpi="300")

for i in range(len(path) - 1):
    node1 = path[i]
    node2 = path[i+1]
    graph.node(node1, label=path[i], shape="box", color="#F3BE26")
    graph.node(node2, label=path[i + 1], shape="box", color = "#F3BE26")
    graph.edge(node1, node2)
    
# saves graph to anomaly_test.png
graph.render("smth3")

# use display func from IPython.display to render graph in juyter

'smth3.png'

In [129]:
import graphviz as gv
graph = gv.Digraph(format="png")
graph.attr(dpi="500")

In [126]:
arrcaseun['Activity'][arrcaseun.index[0:5]]

0    [Заказ на поставку создан, Заказ на поставку и...
1    [Создание Заявки, Заявка изменена дата доставк...
2    [Заказ на поставку создан, Заказ на поставку с...
3    [Создание Заявки, Заявка изменена дата доставк...
4    [Создание Заявки, Заявка изменена дата доставк...
Name: Activity, dtype: object

In [130]:
for path in (arrcaseun['Activity'][arrcaseun.index[0:5]]):
    for j in range(len(path)-1):
        node1 = path[j]
        node2 = path[j + 1]
        graph.node(node1, label=path[j], shape="circle", color="#F3BE26")
        graph.node(node2, label=path[j + 1], shape="circle", color = "#F3BE26")
        graph.edge(node1, node2)

In [131]:
graph.render("finalgraph")

'finalgraph.png'